In [102]:
# %%writefile Factor_Cacher.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd
import numpy as np

import tools.Sample_Tools as smpl
import base.JuUnits as ju

__cache_dir__ = ju.get_root_path()+'/cache.feather/'
__cache_type__ = '.feather'

# df = smpl.get_sample_by_zs(name='沪深300', end='2021-11-28', gap=2500,  only_main=True, filter_st=True).data
# df.to_parquet(__cache_dir__+'test.parquet')

def file_path(file_name):
    return __cache_dir__ + file_name + __cache_type__

def is_cache_exist(name):
    return os.path.exists(file_path(name))

def save_cache(name, data):
    df.reset_index().to_feather(file_path(name))
    
def load_cache(name, to_series=False):
    assert is_cache_exist(name),'cache not exist, create first'
    df = pd.read_feather(file_path(name)).set_index(['date','code'])
    if to_series:
        df =  df.squeeze()
    return df

def load_cache_adv(name, start, end, to_series=False):
    assert is_cache_exist(name),'cache not exist, create first'
        
    st = pd.Timestamp(start) 
    en = pd.Timestamp(end) 

    df = pd.read_feather(file_path(name)).set_index(['date','code'])[st:en]
    date_idx = df.index.get_level_values(0).unique()
    if not (st >= date_idx.min() and en <= date_idx.max()):
        info = 'date range out of cache[{},{}]'.format(
            date_idx.min().strftime('%Y-%m-%d %H:%M:%S'), 
            date_idx.max().strftime('%Y-%m-%d %H:%M:%S')
        )
        raise BaseException(info)
    
    if to_series:
        df =  df.squeeze()
    return df

In [85]:
'date range out of cache[{},{}]'.format(x.min().strftime('%Y-%m-%d %H:%M:%S'), x.max().strftime('%Y-%m-%d %H:%M:%S'))

'date range out of cache[2011-08-15 00:00:00,2021-11-26 00:00:00]'

In [31]:
%%time
df.reset_index().to_feather(__cache_dir__+'test.feather')

Wall time: 71.8 ms


In [107]:
# df['open'].reset_index().to_feather(__cache_dir__+'test_s.feather')
# pd.read_feather(__cache_dir__+'test_s.feather').set_index(['date','code']).squeeze()
# df.squeeze()


In [114]:
%%time
load_cache_adv('test','2011-11-15','2021-11-15')

Wall time: 119 ms


open    high     low   close     volume        amount
date       code                                                           
2011-11-15 000001   16.68   16.72   16.38   16.47   217088.0  3.579836e+08
           000002    7.60    7.60    7.51    7.55   174771.0  1.319559e+08
           000063   18.71   19.16   18.60   19.02   162587.0  3.081418e+08
           000066    6.52    6.79    6.40    6.66   283417.0  1.865707e+08
           000069    7.05    7.09    6.97    7.03   114751.0  8.049150e+07
...                   ...     ...     ...     ...        ...           ...
2021-11-15 603899   61.15   61.91   61.02   61.39    29260.0  1.796513e+08
           603939   48.24   48.81   47.81   48.50    36020.0  1.743625e+08
           603986  163.01  168.98  163.00  166.78   190361.0  3.164767e+09
           603993    5.57    5.60    5.46    5.57  1273340.0  7.039971e+08
           605499  176.00  186.22  175.11  182.72    12252.0  2.231651e+08

[511377 rows x 6 columns]

In [79]:
st = pd.Timestamp('2011-11-15') 
en = pd.Timestamp('2021-11-15') 

x = df.index.get_level_values(0).unique()
# st > x.min() and en < x.max()

df[st:en]

open    high     low   close     volume        amount
date       code                                                           
2011-11-15 000001   16.68   16.72   16.38   16.47   217088.0  3.579836e+08
           000002    7.60    7.60    7.51    7.55   174771.0  1.319559e+08
           000063   18.71   19.16   18.60   19.02   162587.0  3.081418e+08
           000066    6.52    6.79    6.40    6.66   283417.0  1.865707e+08
           000069    7.05    7.09    6.97    7.03   114751.0  8.049150e+07
...                   ...     ...     ...     ...        ...           ...
2021-11-15 603899   61.15   61.91   61.02   61.39    29260.0  1.796513e+08
           603939   48.24   48.81   47.81   48.50    36020.0  1.743625e+08
           603986  163.01  168.98  163.00  166.78   190361.0  3.164767e+09
           603993    5.57    5.60    5.46    5.57  1273340.0  7.039971e+08
           605499  176.00  186.22  175.11  182.72    12252.0  2.231651e+08

[511377 rows x 6 columns]